### Chamada de bibliotecas

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

# ROTEIRO PARA LINKAGE

### Lendo e visualizando as bases

In [84]:
datasetA = spark.read.csv('dataset_a_v3.csv', header=True, sep=';')
datasetB = spark.read.csv('dataset_b_v3.csv', header=True, sep=';')

In [85]:
print datasetA.count()
print datasetB.count()
datasetA.limit(5).show()
datasetB.limit(5).show()

100
20
+-----+--------------------+---------+------+--------------------+--------+---------------+
|cod_a|              nome_a|     dn_a|sexo_a|               mae_a|cidade_a|primeiro_nome_a|
+-----+--------------------+---------+------+--------------------+--------+---------------+
|    1|EDSON GOMES    DO...|1-01-2007|     1|WEDILAINE VIEIRA ...|  280030|          EDSON|
|    2|ALESSANDRA KAUANÈ...|1-02-2008|     2|VITORIA LUCIA AMO...|  280740|     ALESSANDRA|
|    3|DAVI GONÇALVES DA...|1-04-2007|     1| VILMA GOMES MOREIRA|  280030|           DAVI|
|    4|ALISON DE JESUS T...|1-05-2007|     2|VERA LUCIA FRANCI...|  280030|         ALISON|
|    5|DANNYEL COSTA DE ...|1-12-2006|     1|VANILDA TRINDADE ...|  280030|        DANNYEL|
+-----+--------------------+---------+------+--------------------+--------+---------------+

+-----+--------------------+----------+------+--------------------+--------+---------------+
|cod_b|              nome_b|      dn_b|sexo_b|               mae_b|cida

### Criando colunas com codigos foneticos

In [86]:
import jellyfish

In [87]:
def criaMetaphone(col):
    return jellyfish.metaphone(col)
udf_criaMetaphone = F.udf(criaMetaphone, StringType())

In [89]:
datasetA = datasetA.withColumn('phonetic_nome_a', udf_criaMetaphone(F.col('nome_a')))
datasetA = datasetA.withColumn('phonetic_mae_a', udf_criaMetaphone(F.col('mae_a')))
datasetB = datasetB.withColumn('phonetic_nome_b', udf_criaMetaphone(F.col('nome_b')))
datasetB = datasetB.withColumn('phonetic_mae_b', udf_criaMetaphone(F.col('mae_b')))

datasetA.limit(3).show()
datasetB.limit(3).show()

+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+
|cod_a|              nome_a|     dn_a|sexo_a|               mae_a|cidade_a|primeiro_nome_a|     phonetic_nome_a|  phonetic_mae_a|
+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+
|    1|EDSON GOMES    DO...|1-01-2007|     1|WEDILAINE VIEIRA ...|  280030|          EDSON|    ETSN KMS TS SNTS|     WTLN FR BSR|
|    2|ALESSANDRA KAUANÈ...|1-02-2008|     2|VITORIA LUCIA AMO...|  280740|     ALESSANDRA|ALSNTR KN SS TS SNTS|FTR LX AMRM T SS|
|    3|DAVI GONÇALVES DA...|1-04-2007|     1| VILMA GOMES MOREIRA|  280030|           DAVI|     TF KNKLFS T  RX|     FLM KMS MRR|
+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+

+-----+--------------------+----------+------+--------------------+--------+-------------

### Criando coluna com último nome

In [92]:
def criaUltimoNome(col):
    return col.split(' ')[-1]
udf_criaUltimoNome = F.udf(criaUltimoNome, StringType())

In [93]:
datasetA = datasetA.withColumn('ultimo_nome_a', udf_criaUltimoNome(F.col('nome_a')))
datasetB = datasetB.withColumn('ultimo_nome_b', udf_criaUltimoNome(F.col('nome_b')))

datasetA.limit(3).show()
datasetB.limit(3).show()

+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+-------------+
|cod_a|              nome_a|     dn_a|sexo_a|               mae_a|cidade_a|primeiro_nome_a|     phonetic_nome_a|  phonetic_mae_a|ultimo_nome_a|
+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+-------------+
|    1|EDSON GOMES    DO...|1-01-2007|     1|WEDILAINE VIEIRA ...|  280030|          EDSON|    ETSN KMS TS SNTS|     WTLN FR BSR|       SANTOS|
|    2|ALESSANDRA KAUANÈ...|1-02-2008|     2|VITORIA LUCIA AMO...|  280740|     ALESSANDRA|ALSNTR KN SS TS SNTS|FTR LX AMRM T SS|       SANTOS|
|    3|DAVI GONÇALVES DA...|1-04-2007|     1| VILMA GOMES MOREIRA|  280030|           DAVI|     TF KNKLFS T  RX|     FLM KMS MRR|        ROCHA|
+-----+--------------------+---------+------+--------------------+--------+---------------+--------------------+----------------+-------

### Separando atributos para linkage

In [94]:
datasetA = datasetA.select(['cod_a', 'dn_a', 
                            'sexo_a', 'cidade_a', 
                            'primeiro_nome_a', 'ultimo_nome_a', 
                            'phonetic_nome_a', 'phonetic_mae_a'])

datasetB = datasetB.select(['cod_b', 'dn_b', 
                            'sexo_b', 'cidade_b', 
                            'primeiro_nome_b', 'ultimo_nome_b', 
                            'phonetic_nome_b', 'phonetic_mae_b']) 

### Criando dataset de comparação

In [96]:
dataset_linkage = datasetA.crossJoin(datasetB)

In [97]:
dataset_linkage.count()
dataset_linkage.limit(5).show()

+-----+---------+------+--------+---------------+-------------+----------------+--------------+-----+----------+------+--------+---------------+-------------+--------------------+------------------+
|cod_a|     dn_a|sexo_a|cidade_a|primeiro_nome_a|ultimo_nome_a| phonetic_nome_a|phonetic_mae_a|cod_b|      dn_b|sexo_b|cidade_b|primeiro_nome_b|ultimo_nome_b|     phonetic_nome_b|    phonetic_mae_b|
+-----+---------+------+--------+---------------+-------------+----------------+--------------+-----+----------+------+--------+---------------+-------------+--------------------+------------------+
|    1|1-01-2007|     1|  280030|          EDSON|       SANTOS|ETSN KMS TS SNTS|   WTLN FR BSR|    1|09/13/2007|     1|  280030|            EDU|      TAVACHO|          ET PRR TFX|    SMN ANTN RTRKS|
|    1|1-01-2007|     1|  280030|          EDSON|       SANTOS|ETSN KMS TS SNTS|   WTLN FR BSR|    2|05/17/2007|     1|  280030|           JOSE|       SANTOS| JS IKR SNTN TS SNTS|      NKLN FTM MXT|
|    

### Criando função de comparação

In [109]:
def compare(cod_a, dn_a, sexo_a, cidade_a, primeiro_nome_a, ultimo_nome_a, phonetic_nome_a, phonetic_mae_a,
           cod_b, dn_b, sexo_b, cidade_b, primeiro_nome_b, ultimo_nome_b, phonetic_nome_b, phonetic_mae_b):
    sim = 0
    
    # Comparando atributos nominais
    sim_nominais = jellyfish.jaro_winkler(unicode(primeiro_nome_a), unicode(primeiro_nome_b))
    sim_nominais += jellyfish.jaro_winkler(unicode(ultimo_nome_a), unicode(ultimo_nome_b))
    sim_nominais += jellyfish.jaro_winkler(unicode(phonetic_nome_a), unicode(phonetic_nome_b))
    sim_nominais += jellyfish.jaro_winkler(unicode(phonetic_mae_a), unicode(phonetic_mae_b))
    
    # Comparando categorias
    # Note que Hamming é uma distancia, então para saber a similiarade, precisamos
    # encontrar o complemento da medida. 
    sim_cat = 1 - (jellyfish.hamming_distance(unicode(sexo_a), unicode(sexo_b)))
    sim_cat += 1 - (jellyfish.hamming_distance(unicode(dn_a), unicode(sexo_b)))
    sim_cat += 1 - (jellyfish.hamming_distance(unicode(cidade_a), unicode(cidade_b)))
    
    # Media aritmetica simples
    sim = str(abs(float(sim_nominais + sim_cat)/7))
    
    return sim
udf_compare = F.udf(compare, StringType())

### Rodando comparação

In [110]:
result_linkage = dataset_linkage.withColumn('similaridade', udf_compare(F.col('cod_a'), F.col('dn_a'), F.col('sexo_a'), F.col('cidade_a'), F.col('primeiro_nome_a'), F.col('ultimo_nome_a'), F.col('phonetic_nome_a'), F.col('phonetic_mae_a'),
                                                                       F.col('cod_b'), F.col('dn_b'), F.col('sexo_b'), F.col('cidade_b'), F.col('primeiro_nome_b'), F.col('ultimo_nome_b'), F.col('phonetic_nome_b'), F.col('phonetic_mae_b')))

In [111]:
result_linkage.select(['cod_a', 'cod_b', 'similaridade']).show()

+-----+-----+--------------+
|cod_a|cod_b|  similaridade|
+-----+-----+--------------+
|    1|    1|0.371142547928|
|    1|    2|0.313484739988|
|    1|    3|0.311724386724|
|    1|    4|0.404577234934|
|    1|    5|0.378786160929|
|    1|    6|0.971446608947|
|    1|    7|0.431389311746|
|    1|    8| 0.34897443826|
|    1|    9|0.739332096475|
|    1|   10|0.575290582433|
|    1|   11|0.664727709266|
|    1|   12|0.697595856524|
|    1|   13|0.436555403661|
|    1|   14|0.846889221889|
|    1|   15|0.712662337662|
|    1|   16| 0.34705988456|
|    1|   17|0.487968975469|
|    1|   18|0.688083718336|
|    1|   19|0.407490128919|
|    1|   20|0.713810726311|
+-----+-----+--------------+
only showing top 20 rows



## EXERCÍCIOS DE FIXAÇÃO

##### Desafio 1: Deduplique a base de linkage para encontrar os melhores resultados para cada registro do datasetB
##### Desafio 2: Separe a data em três colunas diferentes (dia, mês e ano) e reimplemente a função 'compare' para que a distancia de hamming participe do calculo da similaridade
##### Desafio 3: Quais os atributos mais importantes a serem comparados? Ao invés de uma média aritmética simples, implemente uma média aritmética ponderada dando pesos diferentes para cada atributos.
##### Desafio 4: Explore novas similaridades. Faça uso de diferentes medidas para identidficar as que melhor se encaixam nesse par de datasets. ##### Desafio 5: Leia um pouco sobre penalidades, um decréscimo que se dá à similaridade quando um valor nulo participa da comparação. 
##### Desafio 6: Leia um pouco sobre avaliação de acurácia e faça uma planilha tabulando os resultados de sensibilidade, especificidade e precisão para todos os seus experimentos. 
##### Desafio 7: Tem como melhorar essa implementação? Tente otimizar essa implementação usando os conceitos discutidos em sala. 